# Train YOLO object detector with Turi Create

In [11]:
import os, sys, math
import pandas as pd
import turicreate as tc

ModuleNotFoundError: No module named 'turicreate'

Helper code for loading the CSV file and combining it with an SFrame. We only keep the images that we have annotations for.

In [5]:
def load_images_with_annotations(images_dir, annotations_file):
    # Load the images into a Turi SFrame.
    data = tc.image_analysis.load_images(images_dir, with_path=True)
    
    # Load the annotations CSV file into a Pandas dataframe.
    csv = pd.read_csv(annotations_file)

    # Loop through all the images and match these to the annotations from the
    # CSV file, if annotations are available for the image.
    all_annotations = []
    for i, item in enumerate(data):
        # Grab image info from the SFrame.
        img_path = item["path"]
        img_width = item["image"].width
        img_height = item["image"].height

        # Find the corresponding row(s) in the CSV's dataframe.
        image_id = os.path.basename(img_path)[:-4]
        rows = csv[csv["image_id"] == image_id]

        # Turi expects a list for every image that contains a dictionary for
        # every bounding box that we have an annotation for.
        img_annotations = []
        for i in rows.itertuples():
            img_annotations.append({"coordinates": {"height": (i[5] - i[4]) * img_height, 
                                                    "width": (i[3] - i[2]) * img_width, 
                                                    "x": (i[3] + i[2]) / 2 * img_width, 
                                                    "y": (i[5] + i[4]) / 2 * img_height}, 
                                    "label": i[6]})
        # The CSV file stores the coordinate as numbers between 0 and 1,
        # but Turi wants pixel coordinates in the image.
        
        # A bounding box in Turi is given by a center coordinate and the
        # width and height, we have them as the four corners of the box.

            
            # img_annotations.append({"coordinates": {"height": height, 
            #                                         "width": width, 
            #                                         "x": x, 
            #                                         "y": y}, 
            #                         "label": class_name})

        # If there were no annotations for this image, then append a None
        # so that we can filter out those images from the SFrame.
        if len(img_annotations) > 0:
            all_annotations.append(img_annotations)
        else:
            all_annotations.append(None)

    data["annotations"] = tc.SArray(data=all_annotations, dtype=list)
    return data.dropna()

In [6]:
data_dir = "snacks"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

In [ ]:
train_data = load_images_with_annotations(train_dir, data_dir + "/annotations-train.csv")

In [ ]:
len(train_data)

In [ ]:
train_data.head()

In [ ]:
train_data[0]

Visualize the bounding boxes on top of the training data:

In [ ]:
train_data['image_with_ground_truth'] = tc.object_detector.util.draw_bounding_boxes(
                                            train_data['image'], train_data['annotations'])
train_data.explore()

Train the model. This first downloads the Darknet feature extractor.

In [ ]:
model = tc.object_detector.create(train_data, feature='image', annotations='annotations')

Save the model. Also export to Core ML.

In [ ]:
model.save("SnackDetector.model")

In [ ]:
model.export_coreml("SnackDetector.mlmodel")

Load the model and evaluate it on the test set.

In [ ]:
model = tc.load_model("SnackDetector.model")

In [ ]:
val_data = load_images_with_annotations(val_dir, data_dir + "/annotations-val.csv")
test_data = load_images_with_annotations(test_dir, data_dir + "/annotations-test.csv")

In [ ]:
scores = model.evaluate(test_data)

`model.evaluate()` computes the "average precision" for each class, as well as the overall mean average precision metric. Higher is better.

In [ ]:
scores

Make predictions on the test data. This outputs something like this:

```
[{'confidence': 0.7225357099539148,
  'coordinates': {'height': 73.92794444010806,
                  'width': 90.45315889211807,
                  'x': 262.2198759929745,
                  'y': 155.496952970812},
  'label': 'dog',
  'type': 'rectangle'},
 ...]
```

which is similar to the annotations, but now there is a `confidence` field as well.

In [ ]:
test_data["predictions"] = model.predict(test_data)

In [ ]:
test_data.head()

Visualize the predicted bounding boxes on top of the test set:

In [ ]:
test_data['image_with_predictions'] = tc.object_detector.util.draw_bounding_boxes(
                                           test_data['image'], test_data['predictions'])

In [ ]:
test_data.head()

In [ ]:
test_data.explore()